In [1]:
from requests.auth import HTTPBasicAuth
import requests
import json
import time

#### Deploy a new model

In [2]:
payload = "{\"name\":\"Project06\",\"description\":\"A dummy project\"}"

auth = HTTPBasicAuth('vinaykumar','Testing@1234')

projectURL = "https://zmk.latest.stage.c8y.io/service/mlw/projects"
project = requests.post(projectURL,data=payload,auth=auth)
projectID = json.loads(project.text)['id']

print("project >>>> ",project.status_code)

project >>>>  200


In [3]:
import unittest

class TestCDHIntegration(unittest.TestCase):


    # Test case for Deploying a Model
    def test_DataHub_Success(self):
        url = "https://zmk.latest.stage.c8y.io/service/mlw/projects/"+projectID+"/resources/importFromDatahub/data"
        sqlQuery = {"sql": 'select * from t23897369DataLake."c8y-dremio".t23897369.alarms',
                    "fileName":"myData123"}
        # payload = sqlQuery
        payload = json.dumps(sqlQuery)
        headers = {
        'Content-Type': 'application/json'
        }
        response = requests.post(url,data=payload,headers=headers,auth=auth)
        resp=json.loads(response.text)
        self.assertEqual(response.status_code,200)
        self.assertEqual(resp['type'],"DATAHUB")
        self.assertEqual(resp['name'].split("_")[0],"myData123")
        self.assertEqual(resp['status'],"RUNNING")
        self.assertEqual(resp['projectName'],"Project06")
        self.assertEqual(resp['recurrence'],"ONE_TIME")
        self.assertEqual(resp['properties'][0]['key'],"file_name")
        self.assertEqual(resp['properties'][0]['value'],"myData123")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['status'],"RUNNING")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['type'],"DATAHUB")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['id'].split("_")[1],"DataHub")
        tasksID=resp['id']
        time.sleep(10)
        url='https://zmk.latest.stage.c8y.io/service/mlw/tasks/'+tasksID
        resp=requests.get(url,auth=auth)
        respTasks1=json.loads(resp.text)
        listOfTaskIndi=[i['id'] for i in respTasks1['individualTasks']]
        indTaskID=listOfTaskIndi[0]
        self.assertEqual(respTasks1['individualTasks'][0]['projectID'],projectID)
        urlTask2='https://zmk.latest.stage.c8y.io/service/mlw/tasks/'+tasksID+'/task/'+indTaskID
        resp=requests.get(urlTask2,auth=auth)
        respTasks3=json.loads(resp.content)
        self.assertEqual(respTasks3['status'],'COMPLETED')

    
    def test_DataHub_Failure(self):
        url = "https://zmk.latest.stage.c8y.io/service/mlw/projects/"+projectID+"/resources/importFromDatahub/data"
        sqlQuery = {"sql": 'select * from t23897369DataLake."c8y-dremio".t23897369.alar',
                    "fileName":"myData1234"}
        # payload = sqlQuery
        payload = json.dumps(sqlQuery)
        headers = {
        'Content-Type': 'application/json'
        }
        response = requests.post(url,data=payload,headers=headers,auth=auth)
        resp=json.loads(response.text)
        self.assertEqual(response.status_code,200)
        self.assertEqual(resp['type'],"DATAHUB")
        self.assertEqual(resp['name'].split("_")[0],"myData1234")
        self.assertEqual(resp['status'],"RUNNING")
        self.assertEqual(resp['projectName'],"Project06")
        self.assertEqual(resp['recurrence'],"ONE_TIME")
        self.assertEqual(resp['properties'][0]['key'],"file_name")
        self.assertEqual(resp['properties'][0]['value'],"myData1234")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['status'],"RUNNING")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['type'],"DATAHUB")
        self.assertEqual(resp['individualTasks'][list(resp['individualTasks'].keys())[0]]['id'].split("_")[1],"DataHub")
        tasksID=resp['id']
        time.sleep(10)
        url='https://zmk.latest.stage.c8y.io/service/mlw/tasks/'+tasksID
        resp=requests.get(url,auth=auth)
        respTasks1=json.loads(resp.text)
        listOfTaskIndi=[i['id'] for i in respTasks1['individualTasks']]
        indTaskID=listOfTaskIndi[0]
        self.assertEqual(respTasks1['individualTasks'][0]['projectID'],projectID)
        urlTask2='https://zmk.latest.stage.c8y.io/service/mlw/tasks/'+tasksID+'/task/'+indTaskID
        resp=requests.get(urlTask2,auth=auth)
        respTasks3=json.loads(resp.content)
        self.assertEqual(respTasks3['status'],'FAILED')

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 23.881s

OK
